In [ ]:
%load_ext autoreload

%autoreload 2

In [2]:
from ngram_tools.download_ngrams import download_ngram_files
from ngram_tools.convert_to_jsonl import convert_to_jsonl_files
from ngram_tools.lowercase_ngrams import lowercase_ngrams
from ngram_tools.lemmatize_ngrams import lemmatize_ngrams
from ngram_tools.filter_ngrams import filter_ngrams
from ngram_tools.sort_ngrams import sort_ngrams
from ngram_tools.consolidate_ngrams import consolidate_duplicate_ngrams
from ngram_tools.index_and_create_vocab import index_and_create_vocab_files
from ngram_tools.helpers.verify_sort import check_file_sorted
from ngram_tools.helpers.print_jsonl_lines import print_jsonl_lines

# **Process Unigrams to Generate a Vocabulary File**

## **Goal**: Make a list of the _n_ most common unigrams for later use filtering multigrams

This workflow is resource-intensive and is probably only practical when run on a computing cluster. On my university's High Performance Computing (HPC) cluster, I request the maximum 14 cores (48 logical processors) and 128G of memory and use a 2T fast-I/O NVMe SSD filespace—and I still run up against time and resource limits. I've designed the code to be efficient, although further optimization is surely possible.

The code affords options to conserve resources. Throughout the workflow you can specify `compress=True`, which tells a script to compress its output files. In my experience, there is little downside to using LZ4 compression, since it's very fast and cuts file sizes by about half. Downstream modules will see the `.lz4` extensions and handle the files accordingly. If you know your workflow runs correctly and wish to further conserve space, you can specify `delete_input=True` for many of the scripts; this will delete the source files for a given step once it is complete. The scripts are fairly memory-efficient—with the exception of `sort_ngrams` and `index_and_create_vocab_files`, which sort multiple files in memory at once. When processing multigrams, I've found that allocating more than ~10 workers in these scripts leads to memory exhaustion (with 128G!) and slow processing.

### Download unigrams
Here, I'm using the `download_ngrams` module to fetch unigrams appended with part-of-speech (POS) tags (e.g., `_VERB`). Although you can specify `ngram_type='untagged'`, POS tags are necessary to lemmatize the tokens. Specify the number of parallel processes you wish to use by setting `workers` (the default is all available processors). Note the `repo_release_id` and `repo_corpus_id` parameters; these tell the module which ngram corpus and release to download.

In [5]:
download_ngram_files(
    ngram_size=1,
    ngram_type='tagged',
    repo_release_id='20200217',
    repo_corpus_id='eng',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=27,
    compress=True,
    overwrite=True
)

Start Time:                2025-07-21 12:25:33.347383

Download Info
Ngram repository:          https://storage.googleapis.com/books/ngrams/books/20200217/eng/eng-1-ngrams_exports.html
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download
File index range:          0 to 23
File URLs available:       24
File URLs to use:          24
First file to get:         https://storage.googleapis.com/books/ngrams/books/20200217/eng/1-00000-of-00024.gz
Last file to get:          https://storage.googleapis.com/books/ngrams/books/20200217/eng/1-00023-of-00024.gz
Ngram size:                1
Ngram type:                tagged
Number of workers:         27
Compress saved files:      True
Overwrite existing files:  True



Downloading:   0%|          | 0/24 [00:00<?, ?files/s]


End Time:                  2025-07-21 12:26:03.543466
Total runtime:             0:00:30.196083



### Convert files from TXT to JSONL
This module converts the original unigram files' text data to a more flexible JSON Lines (JSONL) format. Although this increases storage demands, it makes downstream processing more efficient.

In [7]:
convert_to_jsonl_files(
    ngram_size=1,
    ngram_type='tagged',
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=27,
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-07-21 12:26:32.204413

Conversion Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert
File index range:          0 to 23
Files available:           24
Files to use:              24
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download/1-00000-of-00024.txt.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1download/1-00023-of-00024.txt.lz4
Ngram size:                1
Ngram type:                tagged
Number of workers:         27
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Converting:   0%|          | 0/24 [00:00<?, ?files/s]


End Time:                  2025-07-21 12:28:12.188452
Total runtime:             0:01:39.984039



### Make unigrams all lowercase
This module lowercases all characters in the unigrams. Most use cases benefit from this.

In [ ]:
lowercase_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=27,
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-07-21 12:31:58.586521

Lowercasing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase
File index range:          0 to 23
Files available:           24
Files to use:              24
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert/1-00000-of-00024.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/2convert/1-00023-of-00024.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lowercasing:   0%|          | 0/24 [00:00<?, ?files/s]


End Time:                  2025-07-21 12:32:49.420301
Total runtime:             0:00:50.833780



### Lemmatize the unigrams
This module lemmatizes the unigrams—that is, reduces them to their base forms. This is desirable for most use cases. Example: `people_NOUN` ("the people of this land") will be converted to `person` in the output; `people_VERB` ("to people this land") will not. My code uses the [NLTK Lemmatizer](https://www.nltk.org/api/nltk.stem.WordNetLemmatizer.html?highlight=wordnet), which requires requires POS-tagged unigrams. The tags are discarded after lemmatization as they're no longer useful, saving space.

In [11]:
lemmatize_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=27,
    compress=True,
    overwrite=True,
    delete_input=True
)

Start Time:                2025-07-21 12:33:10.922534

Lemmatizing Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase
Output directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize
File index range:          0 to 23
Files available:           24
Files to use:              24
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase/1-00000-of-00024.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/3lowercase/1-00023-of-00024.jsonl.lz4
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True
Delete input directory:    True



Lemmatizing:   0%|          | 0/24 [00:00<?, ?files/s]


End Time:                  2025-07-21 12:34:23.972694
Total runtime:             0:01:13.050160



### Filter the unigrams
This module removes tokens that provide little information about words' semantic context: those that contain numerals (`numerals=True`), nonalphabetic characters (`nonalpha=True`), stopwords (high-frequency, low information tokens like "the" and "into"; `stops=True`), or short words (those below a certain user-specified character count; here, `min_token_length=3`).

You can also specify `min_tokens`—the minumum length of a retained ngram after filtering its tokens. This is mainly intended for use when processing multigrams. However, it's still good to specify `min_tokens=1` for unigrams, as it completely discards the data for any unigram violating our criteria. Empty output files will be deleted.

In [ ]:
filter_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    numerals=True,
    nonalpha=True,
    stops=True,
    min_token_length=3,
    min_tokens=1,
    workers=27,
    compress=True,
    overwrite=True,
    replace_unk=False,
    delete_input=True
)

Start Time:                   2025-07-21 12:34:51.670848

Filtering Info
Input directory:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize
Output directory:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter
File index range:             0 to 23
Files available:              24
Files to use:                 24
First file to get:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize/1-00000-of-00024.jsonl.lz4
Last file to get:             /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/4lemmatize/1-00023-of-00024.jsonl.lz4
Ngram size:                   1
Number of workers:            27
Compress output files:        True
Overwrite existing files:     True
Delete input directory:       True

Filtering Options
Drop stopwords:               True
Drop tokens under:            3 chars
Drop tokens with numerals:    True
Drop non-alphabetic:          True
Drop ngrams under:      

Filtering:   0%|          | 0/24 [00:00<?, ?files/s]


Filtering Results (Dropped)
Stopword tokens:              5609 
Short-word tokens:            22810 
Tokens with numerals:         13740955 
Tokens with non-alpha chars:  2886115
Out-of-vocab tokens:          0
Entire ngrams:                16655489 

End Time:                  2025-07-21 12:35:45.574792
Total runtime:             0:00:53.903944



### Sort and combine the unigram files
This modules creates a single, fully-sorted unigram file out of the filtered files. This is crucial for the next step (ngram consolidation; see below).   

Sorting a giant file is a resource-hungry process and I've tried to implement an efficient approach that leverages parallelism: We first sort the filtered files in parallel using Python's standard sorting algorithm [Timsort](https://en.wikipedia.org/wiki/Timsort); then, we incrementally [heapsort](https://en.wikipedia.org/wiki/Heapsort) the files in parallel until we get down to 2 files. Finally, we heapsort the final 2 files (necessarily using one processor) to arrive at a single combined and sorted unigram file.

In [14]:
sort_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    sort_key='ngram',
    workers=27,
    compress=True,
    overwrite=True,
    sort_order='ascending',
    delete_input=True
)

Start Time:                2025-07-21 12:37:40.299651

Sort Info
Input directory:           /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter
Sorted directory:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/temp
Temp directory:            /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/tmp
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-merged.jsonl.lz4
Files available:           18
First file to get:         /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter/1-00006-of-00024.jsonl.lz4
Last file to get:          /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/5filter/1-00023-of-00024.jsonl.lz4
Files to use:              18
Ngram size:                1
Number of workers:         27
Compress output files:     True
Overwrite existing files:  True
Sort key:                  ngram
Sort order:                ascending
Heap-merge start iter:  

Sorting:   0%|          | 0/18 [00:00<?, ?files/s]


Iteration 1: merging 18 files into 9 chunks using 9 workers.
  9 chunk(s) with 2 file(s)

Iteration 2: merging 9 files into 4 chunks using 4 workers.
  3 chunk(s) with 2 file(s)
  1 chunk(s) with 3 file(s)

Iteration 3: merging 4 files into 2 chunks using 2 workers.
  2 chunk(s) with 2 file(s)

Iteration 4: final merge of 2 files.


Merging:   0%|          | 0/25127726 [00:00<?, ?lines/s]


Merging complete. Final merged file:
/vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-merged.jsonl.lz4

End Time:                  2025-07-21 12:57:48.057527
Total runtime:             0:20:07.757876



### Verify sort [OPTIONAL]
If we want, we can verify that the output file is correctly sorted. If the script outputs `True`, then the file is sorted. Bear in mind that you need to specify the file path manually here; be sure to use the right file extension based on whether `sort_ngrams` was run with `compress=True`.

In [19]:
check_file_sorted(
    input_file=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '1gram_files/6corpus/1gram-merged.jsonl.lz4'
    ),
    field="ngram",
    sort_order="ascending"
)

Lines: 0line [00:00, ?line/s]

Lines: 25127726line [06:14, 67058.01line/s]

The file is sorted.


### Consolidate duplicate unigrams
This module consolidates the sorted unigram file. Lowercasing and lemmatizing produce duplicate unigrams. Now that the file is sorted, we can scan through it and consolidate consecutive idential duplicates. This involves summing their overall and yearly frequencies and documents.

In [21]:
consolidate_duplicate_ngrams(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    lines_per_chunk=500000,
    workers=27,
    compress=True,
    overwrite=True
)

Start Time:                2025-07-21 13:06:53.071867

Consolidation Info
Merged file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-merged.jsonl.lz4
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Temporary directory:       /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/temp_chunks
Ngram size:                1
Number of workers:         48
Compress output files:     True
Overwrite existing files:  True



Created and Sorted: 51 chunks
Merged: 51 chunks


End Time:                  2025-07-21 13:14:57.147241
Total runtime:             0:08:04.075374



### Index unigrams and create vocabulary file
Most use cases will require an indexed list of "valid" (i.e., reasonably common) vocabulary words. This indexing script serves the dual functions of (1) mapping each unigram to an index number (saved in `/6corpus/1gram-corpus-indexed.jsonl`) and (2) culling this file into a vocabulary list consisting of the _n_ most frequent unigrams (saved in `6corpus/1gram-corpus-vocab_list_match.txt`). The vocabulary file provides a critical means of filtering excessively rare words out of the corpus. Unlike files upstream in the workflow, the vocabulary files are not large and don't need to be compressed.

In [22]:
index_and_create_vocab_files(
    ngram_size=1,
    proj_dir='/vast/edk202/NLP_corpora/Google_Books/20200217/eng',
    workers=27,
    compress=True,
    overwrite=True,
    vocab_n=80000
)

Start Time:                2025-07-21 13:16:00.237393

Indexing Info
Corpus file:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-corpus.jsonl.lz4
Indexed file:              /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-indexed.jsonl.lz4
Ngram size:                1
Number of workers:         27
Compress output files:     True
Overwrite existing files:  True

Vocabulary Info
Vocab size (top N):        80000
Match File:                /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-corpus-vocab_list_match.txt
Lookup File:               /vast/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/6corpus/1gram-corpus-vocab_list_lookup.jsonl



Chunking:   0%|          | 0/13499384 [00:00<?, ?lines/s]

Sorting:   0%|          | 0/135 [00:00<?, ?chunks/s]

Merging:   0%|          | 0/13499384 [00:00<?, ?lines/s]

Indexing:   0%|          | 0/13499384 [00:00<?, ?lines/s]


Indexed 13499384 lines.
Final indexed file: 1gram-corpus-indexed.jsonl.lz4
Created vocab_list_match and vocab_list_lookup files for top 80000 ngrams.

End Time:                  2025-07-21 13:29:42.404221
Total runtime:             0:13:42.166828



### Verify indexing [OPTIONAL]
We can verify that the final indexed file looks right.

In [27]:
print_jsonl_lines(
    file_path=(
        '/vast/edk202/NLP_corpora/Google_Books/20200217/eng/'
        '1gram_files/6corpus/1gram-corpus-indexed.jsonl.lz4'
    ),
    start_line=70000,
    end_line=70002,
    parse_json=True
)

Line 70000: {'ngram': 'herve', 'freq_tot': 259695, 'doc_tot': 121103, 'freq': {'1633': 1, '1675': 1, '1677': 1, '1679': 1, '1683': 1, '1686': 1, '1693': 1, '1705': 2, '1723': 1, '1728': 1, '1738': 1, '1739': 1, '1741': 1, '1742': 2, '1746': 1, '1748': 1, '1752': 1, '1754': 2, '1756': 1, '1757': 2, '1758': 1, '1760': 6, '1761': 3, '1762': 1, '1763': 1, '1764': 2, '1765': 2, '1766': 2, '1767': 1, '1769': 6, '1773': 4, '1774': 4, '1775': 1, '1776': 2, '1781': 1, '1782': 7, '1784': 4, '1786': 2, '1788': 2, '1790': 6, '1791': 2, '1792': 3, '1793': 10, '1794': 14, '1795': 7, '1796': 26, '1797': 2, '1798': 6, '1799': 3, '1800': 29, '1802': 10, '1803': 7, '1804': 22, '1805': 66, '1806': 65, '1807': 14, '1808': 95, '1809': 4, '1810': 33, '1811': 30, '1812': 10, '1813': 12, '1814': 16, '1815': 36, '1816': 38, '1817': 89, '1818': 17, '1819': 17, '1820': 27, '1821': 33, '1822': 19, '1823': 16, '1824': 43, '1825': 31, '1826': 21, '1827': 33, '1828': 14, '1829': 27, '1830': 34, '1831': 26, '1832': 3

## Next Steps
If you've gotten this far, you're ready to start pre-processing multigrams using the `workflow_multigrams.ipynb` notebook!